In [39]:
import torch
from transformers import ViTModel, ViTImageProcessor, AutoModelForImageClassification, ViTImageProcessorFast
from PIL import Image
import os
from tqdm import tqdm
import pandas as pd
import numpy

In [35]:
def load_models(modelo):

    if modelo == 'ViT_huge':
        vit_huge_model = ViTModel.from_pretrained('google/vit-huge-patch14-224-in21k')
        vit_huge_feature_extractor = ViTImageProcessor.from_pretrained('google/vit-huge-patch14-224-in21k')
        vit_huge_model.eval()

        return vit_huge_model, vit_huge_feature_extractor

    if modelo == 'ViT_large':
        vit_large_model = ViTModel.from_pretrained('google/vit-large-patch16-224-in21k')
        vit_large_feature_extractor = ViTImageProcessor.from_pretrained('google/vit-large-patch16-224-in21k')
        vit_large_model.eval()

        return vit_large_model, vit_large_feature_extractor
    
    if modelo == 'ViT_base':
        vit_base_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        vit_base_feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
        vit_base_model.eval()
    
        return vit_base_model, vit_base_feature_extractor

    if modelo == 'ViT_small':
        vit_small_model = AutoModelForImageClassification.from_pretrained('WinKawaks/vit-small-patch16-224')
        vit_small_feature_extractor = ViTImageProcessorFast.from_pretrained('WinKawaks/vit-small-patch16-224')
        vit_small_model.eval()

        return vit_small_model, vit_small_feature_extractor

    return 


In [40]:
def feature_extraction(image_path, modelo, feature_extractor):
    image = Image.open(image_path).convert('RGB')
    inputs = feature_extractor(images=image, return_tensors="pt")

    with torch.no_grad():
        outputs = modelo(**inputs)
    
    features = outputs.logits.mean(dim=1).squeeze().numpy()
    return features

In [44]:
def features_to_df(folder_path, modelo, feature_extractor):
    data = []
    
    for subfolder in tqdm(os.listdir(folder_path)):
        subfolder_path = os.path.join(folder_path, subfolder)
    
    if os.path.isdir(subfolder_path):
        for image_file in os.listdir(subfolder_path):
            image_path = os.path.join(subfolder_path, image_file)
            
            if image_file.lower().endswith(('png', 'jpg', 'jpeg')):
                features = feature_extraction(image_path, modelo, feature_extractor)
                    
                data.append([image_path, *features])

    columns = ['image_path'] + [f'feature_{i}' for i in range(len(features))]
    df = pd.DataFrame(data, columns=columns)
    return df

    

In [38]:
model, fe = load_models('ViT_small')

In [45]:
df = features_to_df('./Soybean Seeds/',model, fe)

100%|███████████████████████████████████████████| 5/5 [00:00<00:00, 4744.69it/s]


TypeError: iteration over a 0-d array

In [48]:
image_path= './Soybean_Seeds/Intact_soybeans/1.jpg'
image = Image.open(image_path).convert('RGB')
inputs = fe(images=image, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

last_hidden_state = outputs.last_hidden_state
print(last_hidden_state)
#features = outputs.logits.mean(dim=1).squeeze().numpy()


AttributeError: 'ImageClassifierOutput' object has no attribute 'last_hidden_state'